In [133]:
from google.cloud import vision
from google.cloud import datastore
from google.oauth2 import service_account
import io
import os # for use with setting env variables
import re
from datetime import datetime


In [299]:
""" TODO
* Discount line logic
* Create an interface where the user can manually give an item its category:
    * Remember ID and update the category_id, keep other values as the same
"""

' TODO\n* Discount line logic\n* Create an interface where the user can manually give an item its category:\n    * Remember ID and update the category_id, keep other values as the same\n'

In [161]:
debug = False
developing = True
key_path = "/Volumes/GoogleDrive/My Drive/00. My Documents/03. Internt/24. Expense analyzer/config_files/expense-analyzer-260008-0cac2ecd3671.json"

In [302]:
# Only used in dev environment
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = key_path
credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=["https://www.googleapis.com/auth/cloud-platform"]
)

In [289]:
OVERLAPPING_ALLOCATION_THRESHOLD = 0.3

# The entity kind in datastore to query to find previous assignments
DATASTORE_KIND_CATEGORY_ASSIGNMENT = "category_item_mapping"

In [126]:

def crop_and_rotate(path):
    # do some magic here
    # Identify how scewed the image is
    # Rotate it
    return 1

def detect_text(path):
    """Detects text in the file."""
    client = vision.ImageAnnotatorClient.from_service_account_json(key_path)

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    
    if debug:
        print('Texts:')
        for text in texts:
            print('\n"{}"'.format(text.description))

            vertices = (['({},{})'.format(vertex.x, vertex.y)
                        for vertex in text.bounding_poly.vertices])

            print('bounds: {}'.format(','.join(vertices)))

    return response


In [98]:
#  ### PARSER FOR COOP
def article_lines_coop(response):
    # Locate the relevant range to extract items
    start_y_coordinate = -1     # Determine which point to start extract items
    end_y_coordinate = -1       # Determine which point to stop extract items
    
    receipt_id_and_datetime = ""

    for text in response.text_annotations:
        if len(text.description) > 100:
            # Full text, extract the receipt ID and datetime
            full_text = text.description

            # Start substring from the search term
            search_term = "Salgskvittering"
            start_index = full_text.find(search_term) + len(search_term)

            # End the substring at next newline
            end_index = start_index + full_text[start_index:].find("\n")
            
            # This now contains id, date, time, separated by space
            receipt_id_and_datetime = full_text[start_index:end_index].strip()            
        elif "Salgskv" in text.description:
            start_y_coordinate = max(vertex.y for vertex in text.bounding_poly.vertices)
            if debug:
                print("Found starting point at {}, after text {}".format(start_y_coordinate, text.description))
        elif "Totalt" in text.description:
            end_y_coordinate = min(vertex.y for vertex in text.bounding_poly.vertices)
            if debug:
                print("Found ending point at {}, after text {}".format(end_y_coordinate, text.description))

    # Iterate through all lines, extract only those with item y coordinate larger than start and smaller than end
    relevant_items = []
    for text in response.text_annotations:
        if text.bounding_poly.vertices[0].y > start_y_coordinate and text.bounding_poly.vertices[0].y < end_y_coordinate:
            # print("Found an item line!: {}".format(text.description))
            relevant_items.append(text)
            
    return relevant_items, receipt_id_and_datetime


# Key = line_number, value = item
# Idea: For each bounding box, calculate the mid y coordinate. If this coordinate is inside the bounding box of
# another, then these are on the same line.
def allocate_lines_coop(items):
    """
    :param items: input is a list of relevant text boxes from Google vision, containing the text found and bounding polygon
    :return: returns a dictionary, with all items allocated to a line_id containing all elements on that same line, sorted by their x-coordinates
    """

    receipt_lines = {}
    # Key is the line number
    # Each value has the format [item]

    # Loop over all found text boxes
    for item in items:
        y_first_coor = item.bounding_poly.vertices[0].y
        y_fourth_coor = item.bounding_poly.vertices[3].y
        y_mean_coor = (y_first_coor + y_fourth_coor)/2
        height = y_fourth_coor - y_first_coor

        overlap_up = -1
        overlap_down = -1

        if len(receipt_lines) == 0:
            receipt_lines[0] = []
            receipt_lines[0].append(item)

        else:
            inserted = False

            # Loop through all allocated/identified lines
            for line in receipt_lines:
                # See if item belongs to an existing line
                # Compare against y coordinates of first item on line
                first_line_item = receipt_lines[line][0]
                first_line_item_y1 = first_line_item.bounding_poly.vertices[0].y
                first_line_item_y4 = first_line_item.bounding_poly.vertices[3].y


                # if mean coordinate is within min and max of line, add it to the line
                if first_line_item_y1 <= y_mean_coor <= first_line_item_y4:
                    receipt_lines[line].append(item)
                    inserted = True
                    break

                # These are used to calculate overlap between lines
                last_line_item = receipt_lines[line][-1]
                last_line_item_y1 = last_line_item.bounding_poly.vertices[0].y
                last_line_item_y4 = last_line_item.bounding_poly.vertices[3].y

                # Calculate a match % against each other line, to see if picture is slightly squished
                # Calculate against the item to the far right in the current line
                if last_line_item_y1 <= y_first_coor <= last_line_item_y4:
                    # Some overlap detected. item is below the line in comparison
                    overlap_down = float(last_line_item_y4 - y_first_coor) / height
                    if debug:
                        print("Found {}% overlap under between text {} and {} on line {}".format(overlap_down, item.description, first_line_item.description, line))

                if last_line_item_y1 <= y_fourth_coor <= last_line_item_y4:
                    # Some overlap detected. item is above the line in comparison
                    overlap_up = float(y_fourth_coor - last_line_item_y1) / height
                    if debug:
                        print("Found {}% overlap over between text {} and {} (first item) on line {}".format(overlap_up, item.description, first_line_item.description, line))

                # If any of the matches are above X%, allocate it to that line
                if overlap_down > OVERLAPPING_ALLOCATION_THRESHOLD or overlap_up > OVERLAPPING_ALLOCATION_THRESHOLD:
                    receipt_lines[line].append(item)
                    inserted = True
                    break;

            # No match found against previous lines. Create a new line
            if not inserted:
                new_line_num = len(receipt_lines)
                receipt_lines[new_line_num] = []
                receipt_lines[new_line_num].append(item)

    # Sort each line by the x coordinates
    for line in receipt_lines:
        receipt_lines[line].sort(key=lambda item: item.bounding_poly.vertices[0].x)

    return receipt_lines


def lines_to_text(receipt_lines):
    """
    :param receipt_lines: takes a dictionary as input, where the key is a line_id and the value are objects containing the element text and bounding polygon
    :return: A list of text strings concatenated for each line, instead of google vision objects
    """
    receipt_text = []
    for line in receipt_lines:
        text = ""
        for item in receipt_lines[line]:
            text += " " + item.description
        receipt_text.append(text.lower().strip())
    return receipt_text

In [162]:
def query_preparation(receipt_text):
    """
    :param receipt_text: list of text strings containing article text and price
    :return: a list of multiple tuples, consisting of article type, text and price
    """

    """
    Regex:
    "\d+" matches one or more digits
    "." followed by any charcter
    "\d+" one or more digits
    "$" at the end of the line
    """
    price_pattern = "(\d+.\d+)$"
    discont_pattern = ""
    articles_querified = []

    for article in receipt_text:
        if debug:
            print("------------")
            print(article)

        if "rabatt" in article:
            # TODO: x = re.spltt(discount_pattern, article)
            # articles.append(["discount", x[0], x[1]])
            
            # A line starting with "Rabatt" belongs to the line before.
            if developing:
                print("Found a discount line")
                print(article.split(" "))

        elif "antall" in receipt_text:
            # Do something TODO
            if developing:
                print("jadajada")

        elif "artikler" not in article:
            x = re.split(price_pattern, article, 2)
            
            # Element 3, index 2, is always empty string
            if len(x) == 3:
                # actual article
                item = x[0].strip()
                price = x[1].strip()             
                articles_querified.append([item, price])
                if debug:
                    print("len was 3")
                    print("Appending item '{}' with price '{}'. Full split is '{}'".format(item, price, x))
            else:
                # Typically weight times price per kg.
                if developing:
                    print("Unparsable line: {}".format(article))
        else:
            if developing:
                print("Unparsable line 2: {}".format(article))
    return articles_querified

In [293]:
def fetch_item_category(item_name):
    """ Search through similar items and reuse their category
        Give it a category of 0 if it not seen before
    """
    
    query = datastore_client.query(kind=DATASTORE_KIND_CATEGORY_ASSIGNMENT)
    #query.add_filter("item_name", "=", item_name)
    
    # Create a filter on the key
    first_key = datastore_client.key(DATASTORE_KIND_CATEGORY_ASSIGNMENT, item_name)
    query.key_filter(first_key, '=')
    
    # Fetch only one result
    q_result = query.fetch(limit=1)
    
    category_id = 0
    for res in q_result:
        category_id = res["cat_id"]
    
    return category_id

In [279]:
def missing_category(max_records):
    """
    :param max_records (int): number of items to return
    :return: a number of unclassified items from the category assignment registry
    """
    
    query = datastore_client.query(kind=DATASTORE_KIND_CATEGORY_ASSIGNMENT)
    query.add_filter("cat_id", "=", -1)
    
    # Fetch first X results
    q_result = query.fetch(limit=max_records)
    
    return list(q_result)

In [213]:
def upload_categories():
    """
    Uploades categories based on excel file
    
    """
    import pandas as pd
    filepath = "../data/varekategorier.xlsx"
    cats = pd.read_excel(filepath)
    
    # Instantiates a client
    datastore_client = datastore.Client(
        credentials=credentials
    )

    # The kind for the new entity
    kind = 'category'

    for cat in cats.iterrows():

        # The Cloud Datastore key for the new entity
        category_id = str(cat[1]["Varegruppe"])
        task_key = datastore_client.key(category_id)

        # Prepares the new entity
        task = datastore.Entity(key=task_key)
        task['cat_name'] = cat[1]["VAREGRUPPE NAVN"]

        if debug:
            print("Writting to datastore:",task)

        # Saves the entity
        datastore_client.put(task)

In [287]:
def writeToDatastore(articles_querified, added_by, trans_datetime, receipt_id):
    """
    :param articles_querified: list of text strings containing article text and price
    :return: none
    """
    
    # Instantiates a client
    datastore_client = datastore.Client(
        credentials=credentials
    )

    # The kind for the new entity
    kind = 'transaction'
    
    # Registered datetime
    now = datetime.now()
    
    # Loop over all articles and insert one by one
    for article in articles_querified:
        #article = articles_querified[0]
        item = article[0]
        price = article[1]

        category_id = fetch_item_category(item)
        if debug:
            print("Assignning category {} to item {}".format(category_id, item))

        if category_id == 0:
            # TODO: THIS IS A NEW ITEM WE HAVE NOT SEEN BEFORE. ADD IT WITH -1 IN CATEGORY_MAPPING_ENTITIES
            cat_task_key = datastore_client.key(DATASTORE_KIND_CATEGORY_ASSIGNMENT, item)
            cat_task = datastore.Entity(key=cat_task_key)
            cat_task["cat_id"] = -1
            datastore_client.put(cat_task)
            print('Saved {}: {}'.format(cat_task.key.name, cat_task['cat_id']))
            
        # The Cloud Datastore key for the new entity. Creating with partial key
        task_key = datastore_client.key(kind)

        # Prepares the new entity
        task = datastore.Entity(key=task_key)
        task['added_by'] = added_by
        task['cat_id'] = category_id
        task['discount_amt'] = 0 # Missing
        task['discount_type'] = 0 # Missing
        task['item_id'] = 0 # Missing
        task['item_name'] = item
        task['price_gross'] = 0 # Missing
        task['price_net'] = price
        task['registered_datetime'] = now
        task['trans_date'] = trans_datetime
        task['receipt_id'] = receipt_id

        if debug:
            print("Writting to datastore:",task)

        # Saves the entity
        datastore_client.put(task)

        print('Saved {}: {}'.format(task.key.name, task['item_name']))

In [25]:
path = r"C:\Users\NO007454\Documents\03. Internt\24. Expense analyzer\test_images\IMG_1010.JPEG"
path = r"C:\Users\NO007454\Documents\03. Internt\24. Expense analyzer\test_images\IMG_1012.JPEG"
path = "/Volumes/GoogleDrive/My Drive/00. My Documents/03. Internt/24. Expense analyzer/test_images/IMG_1010.JPEG"


In [62]:
response = detect_text(path)

In [99]:
relevant_items, receipt_id_and_datetime = article_lines_coop(response)
receipt_lines = allocate_lines_coop(relevant_lines)
actual_lines = lines_to_text(receipt_lines)

In [165]:
articles_querified = query_preparation(actual_lines)

Found a discount line
['rabatt:', 'nok', '15.00', '(37.6%', 'av', '39.90)']
Found a discount line
['rabatt:', 'nok', '9.87', '(30%', 'av', '32.90)']
Unparsable line: 0.080 kg 40.90 kr/kg
Unparsable line: 0.242 kg 59.90 kr/kg
Unparsable line: tomater ( 24.90)
Unparsable line: 0.186 kg 24.90 kr/kg 7.98)
Found a discount line
['rabatt:', 'nok', '3.35', '(42%', 'av']
Unparsable line 2: artikler ) 324.53


In [166]:
# Extract the receipt id and datetime before saving to datastore
receipt_id = receipt_id_and_datetime.split(" ")[0].strip()
receipt_date = receipt_id_and_datetime.split(" ")[1].strip()
datetime_object = datetime.strptime(receipt_date, '%m.%d.%Y')

In [300]:
writeToDatastore(articles_querified, "testuser", datetime_object, receipt_id)

Saved None: agurk stk
Saved None: avokado modnet 2pk
Saved None: bærepose mega
Saved None: coop bomullspinner
Saved None: coop kjøttdeig
Saved None: coop maiskorn 3pk
Saved None: gilde krydderskinke
Saved None: hvitløk 100g
Saved None: jozo kvernsalt 500g
Saved None: lime kg
Saved None: mack is.lit. 0.5l bx
Saved None: pant
Saved None: oep tortillas 8stk
Saved None: q-lettrømme 300g
Saved None: rød paprika
Saved None: s.m sort pepper 22g
Saved None: s.m taco sauce med.
Saved None: spinatsalat vas.200g 22.90
